### Prepare & Select
Prepare an N-qubit GHZ state and select the all ones state.

In [2]:
from circuits.cirq import cirq_prep_select 
from write_qasm import write_qasm

num_qubits = 25
target_state="1" * num_qubits

circuit = cirq_prep_select(num_qubits, target_state=target_state)
filename = f"prep_select_N{num_qubits}_ghz"

write_qasm(circuit, 
circuit_name=filename, 
basis_gates=['rz', 'rx', 'ry', 'h', 'cx'])

### QCNN
N-qubit Quantum convolutional neural network 

In [2]:
from circuits.qiskit import qcnn_circuit
from numpy import log2, ceil

num_qubits = 100
num_layers = int(ceil(log2(num_qubits)))

seed = 12345

circuit = qcnn_circuit(num_qubits, seed=seed)
filename = f"qcnn_N{num_qubits}_{num_layers}layers"

write_qasm(circuit, 
circuit_name=filename, 
basis_gates=['rz', 'rx', 'ry', 'h', 'cx'])

### Parameterized VQE ansatz


In [7]:
from circuits.qiskit import VQE_ansatz

num_qubits = 50
num_layers = num_qubits * 2

circuit = VQE_ansatz(num_qubits, num_layers)
filename = f"VQE_ansatz_N{num_qubits}_{num_layers}layers"

write_qasm(circuit, circuit_name=filename, version='3')

### QAOA neighest neighbors Ising model

In [1]:

from circuits.qiskit import qaoa_ising_ansatz
from write_qasm import write_qasm

num_qubits = 50
num_layers = num_qubits * 3

circuit = qaoa_ising_ansatz(num_qubits, num_layers)
filename = f"QAOA_Ising_ansatz_N{num_qubits}_{num_layers}layers"

write_qasm(circuit, circuit_name=filename, version='3')

In [4]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

def generate_custom_qaoa_circuit(num_qubits, num_layers):
    """
    Generates a custom QAOA ansatz for a nearest-neighbor Ising Hamiltonian.

    Args:
        num_qubits (int): Number of qubits for the Ising Hamiltonian.
        num_layers (int): Number of QAOA layers.

    Returns:
        QuantumCircuit: Custom QAOA ansatz circuit.
    """
    # Initialize a quantum circuit with the required number of qubits
    qc = QuantumCircuit(num_qubits)

    # Define the parameters for each layer (gamma for cost and beta for mixer)
    gamma = [Parameter(f"γ_{i}") for i in range(num_layers)]
    beta = [Parameter(f"β_{i}") for i in range(num_layers)]

    # Create the QAOA circuit layer by layer
    for layer in range(num_layers):
        # Cost Hamiltonian Evolution: Apply ZZ interactions and single-qubit Z rotations
        for i in range(num_qubits):
            # Apply nearest-neighbor ZZ interaction if not at the last qubit
            if i < num_qubits - 1:
                qc.cx(i, i + 1)
                qc.rz(2 * gamma[layer], i + 1)
                qc.cx(i, i + 1)

            # Apply local Z rotation (Z_i) for each qubit
            qc.rz(2 * gamma[layer], i)

        # Mixer Hamiltonian Evolution: Apply RX rotations
        for i in range(num_qubits):
            qc.rx(2 * beta[layer], i)

    return qc

# Example usage
num_qubits = 4
num_layers = 3
custom_qaoa_circuit = generate_custom_qaoa_circuit(num_qubits, num_layers)

# Print the generated QAOA circuit
print(custom_qaoa_circuit.decompose().draw())


global phase: -7*γ_0 - 7*γ_1 - 7*γ_2
                            ┌───────────┐┌────────────┐                  »
q_0: ──■─────────────────■──┤ U1(2*γ_0) ├┤ R(2*β_0,0) ├──────────────────»
     ┌─┴─┐┌───────────┐┌─┴─┐└───────────┘└────────────┘     ┌───────────┐»
q_1: ┤ X ├┤ U1(2*γ_0) ├┤ X ├──────■──────────────────────■──┤ U1(2*γ_0) ├»
     └───┘└───────────┘└───┘    ┌─┴─┐    ┌───────────┐ ┌─┴─┐└───────────┘»
q_2: ───────────────────────────┤ X ├────┤ U1(2*γ_0) ├─┤ X ├──────■──────»
                                └───┘    └───────────┘ └───┘    ┌─┴─┐    »
q_3: ───────────────────────────────────────────────────────────┤ X ├────»
                                                                └───┘    »
«                                                   ┌───────────┐┌────────────┐»
«q_0: ────────────────■─────────────────────■───────┤ U1(2*γ_1) ├┤ R(2*β_1,0) ├»
«     ┌────────────┐┌─┴─┐┌───────────┐    ┌─┴─┐     └───────────┘└────────────┘»
«q_1: ┤ R(2*β_0,0) ├┤ X ├┤ U1(2*γ_1) ├────┤ X